In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45044
2,Huelva,Confirmados PDIA 14 días,414
3,Huelva,Tasa PDIA 14 días,"80,6750199738878"
4,Huelva,Confirmados PDIA 7 días,173
5,Huelva,Tasa PDIA 7 dias,"33,71202525478886"
6,Huelva,Total Confirmados,45245
7,Huelva,Curados,42673
8,Huelva,Fallecidos,411


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45044.0


/tmp/ipykernel_3712/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12463.0


/tmp/ipykernel_3712/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3712/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3712/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3712/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3196 personas en los últimos 7 días 

Un positivo PCR cada 1754 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45044.0,173.0,414.0,513170.0,33.712025,80.675020,34.0
Huelva-Costa,26663.0,111.0,289.0,289548.0,38.335613,99.810739,17.0
Condado-Campiña,13975.0,49.0,82.0,156231.0,31.363814,52.486382,15.0
Cartaya,1948.0,23.0,73.0,20083.0,114.524722,363.491510,6.0
Huelva (capital),12463.0,45.0,82.0,143837.0,31.285413,57.008975,5.0
Paterna del Campo,201.0,8.0,8.0,3457.0,231.414521,231.414521,5.0
Almonte,2265.0,11.0,23.0,24507.0,44.885135,93.850737,3.0
Ayamonte,2042.0,19.0,42.0,21104.0,90.030326,199.014405,3.0
Palos de la Frontera,1013.0,12.0,12.0,11742.0,102.197241,102.197241,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,33.0,1.0,2.0,396.0,252.525253,505.050505,0.0
Cartaya,1948.0,23.0,73.0,20083.0,114.524722,363.491510,6.0
Aroche,128.0,1.0,9.0,3054.0,32.743942,294.695481,0.0
Sanlúcar de Guadiana,35.0,1.0,1.0,403.0,248.138958,248.138958,0.0
Higuera de la Sierra,65.0,1.0,3.0,1291.0,77.459334,232.378002,0.0
Paterna del Campo,201.0,8.0,8.0,3457.0,231.414521,231.414521,5.0
Cumbres Mayores,135.0,2.0,4.0,1749.0,114.351058,228.702115,0.0
Isla Cristina,3104.0,11.0,47.0,21393.0,51.418688,219.698032,2.0
Ayamonte,2042.0,19.0,42.0,21104.0,90.030326,199.014405,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,128.0,1.0,9.0,3054.0,32.743942,294.695481,0.0,0.111111
San Juan del Puerto,758.0,1.0,6.0,9411.0,10.625863,63.755180,0.0,0.166667
Isla Cristina,3104.0,11.0,47.0,21393.0,51.418688,219.698032,2.0,0.234043
Aracena,668.0,1.0,4.0,8255.0,12.113870,48.455482,0.0,0.250000
Cartaya,1948.0,23.0,73.0,20083.0,114.524722,363.491510,6.0,0.315068
Aljaraque,1612.0,1.0,3.0,21474.0,4.656794,13.970383,0.0,0.333333
Valverde del Camino,833.0,1.0,3.0,12750.0,7.843137,23.529412,0.0,0.333333
Higuera de la Sierra,65.0,1.0,3.0,1291.0,77.459334,232.378002,0.0,0.333333
Sierra de Huelva-Andévalo Central,3984.0,13.0,38.0,67391.0,19.290410,56.387351,2.0,0.342105
